# 环境配置

In [3]:
#安装mindnlp 0.4.0套件
!pip install mindnlp==0.4.0
!pip uninstall soundfile -y
!pip install download
!pip install jieba
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.3.1/MindSpore/unified/aarch64/mindspore-2.3.1-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
DEPRECATION: moxing-framework 2.1.16.2ae09d45 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
DEPRECATION: moxing-framework 2.1.16.2ae09d45 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
DEPRECATION: moxing-framework 2.1.16.2ae09d45 has a non-standard version number. pip 24.0 wil

In [1]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn

from mindnlp.dataset import load_dataset

from mindnlp.engine import Trainer

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.311 seconds.
Prefix dict has been built successfully.


In [2]:
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']

In [3]:
imdb_train.get_dataset_size()

25000

In [4]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=4, shuffle=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'attention_mask': (None, 0)})

    return dataset

In [5]:
from mindnlp.transformers import OpenAIGPTTokenizer
# tokenizer
gpt_tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [6]:
#这行代码是为了方便体验流程，把原本数据集的十分之一拿出来体验训练和评估,体验该完整的数据集，可以将这行代码注释掉
imdb_train, _ = imdb_train.split([0.1, 0.9], randomize=False)

# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

In [7]:
dataset_train = process_dataset(imdb_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, gpt_tokenizer)
dataset_test = process_dataset(imdb_test, gpt_tokenizer)

In [8]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[4, 512], dtype=Int64, value=
 [[  616,  5648,  5973 ... 40480, 40480, 40480],
  [  249, 34531,   256 ... 40480, 40480, 40480],
  [  481,  2502,  7322 ...  3808,   481,  1164],
  [ 1684,   616,   504 ... 40480, 40480, 40480]]),
 Tensor(shape=[4, 512], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 1, 1, 1],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[4], dtype=Int32, value= [0, 0, 0, 0])]

In [9]:
from mindnlp.transformers import OpenAIGPTForSequenceClassification
from mindnlp import evaluate
import numpy as np

# set bert config and define parameters for training
model = OpenAIGPTForSequenceClassification.from_pretrained('openai-gpt', num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)

from mindnlp.engine import TrainingArguments

training_args = TrainingArguments(
    output_dir="gpt_imdb_finetune",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    num_train_epochs=1.0,
    learning_rate=2e-5
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics
)


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of OpenAIGPTForSequenceClassification were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 注: 如果想要运行的更快一些，可以在训练时需要V100的算力

In [10]:
trainer.train()

  0%|          | 0/438 [00:00<?, ?it/s]

100%|██████████| 438/438 [03:11<00:00,  2.85it/s]

{'loss': 0.0024, 'learning_rate': 0.0, 'epoch': 1.0}



100%|██████████| 188/188 [00:20<00:00, 18.53it/s]
                                                 
100%|██████████| 438/438 [03:33<00:00,  2.85it/s]
                                                 

{'eval_loss': 5.6351964303757995e-06, 'eval_accuracy': 1.0, 'eval_runtime': 22.1623, 'eval_samples_per_second': 8.483, 'eval_steps_per_second': 1.083, 'epoch': 1.0}


100%|██████████| 438/438 [03:48<00:00,  1.91it/s]

{'train_runtime': 228.9073, 'train_samples_per_second': 7.654, 'train_steps_per_second': 1.913, 'train_loss': 0.0024289760959747175, 'epoch': 1.0}


TrainOutput(global_step=438, training_loss=0.0024289760959747175, metrics={'train_runtime': 228.9073, 'train_samples_per_second': 7.654, 'train_steps_per_second': 1.913, 'train_loss': 0.0024289760959747175, 'epoch': 1.0})

In [11]:
trainer.evaluate(dataset_test)

100%|██████████| 6250/6250 [12:11<00:00,  8.55it/s]


{'eval_loss': 6.262429237365723,
 'eval_accuracy': 0.5,
 'eval_runtime': 732.4071,
 'eval_samples_per_second': 8.534,
 'eval_steps_per_second': 1.068,
 'epoch': 1.0}